### Scraping e análise das maiores empresas do mundo listadas no Global Fortune 500

#### Introdução

O Web Scraping é uma ferramenta importante para obtenção de dados, pois ele torna estas informações acessíveis para todos os tipos de aplicações e análises. É uma técnica utilizada para "minerar" os dados disponíveis nos sites, extraindo deles os dados relevantes para um determinado assunto. Este mesmo processo pode ser realizado manualmente, porém com o Web Scraping é possível automatizar a coleta de uma grande massa de dados e transformá-los em uma estrutura legível para análise em uma curta fração de tempo. 

A utilização de APIs é, na maioria das vezes, a melhor forma de extrair os dados da web, porém quando o site não fornece este serviço o Web Scraping pode ser a alternativa viável.  

O BeautifulSoup e o Selenium são algumas das bibliotecas conhecidas para a aplicação do scraping. O BeautifulSoup é uma biblioteca Python projetada para facilitar a extração de dados nos documentos estruturados em HTML. Atua na manipulação de páginas estáticas, ou seja, não é possível capturar as informações renderizadas via JavaScript. [Documentação Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ptbr/)

O Selenium é um framework voltado à testes de aplicações web pelo browser de forma automatizada. Além do seu uso como ferramenta de testes, é bastante utilizado na coleta de dados ou na criação de rotinas para manipulação de páginas da web. Ele basicamente simula o comportamento do usuário em uma página web. [Documentação Selenium](https://selenium-python.readthedocs.io/)

Neste material, as duas técnicas foram aplicadas, pois a estrutura do web site, onde a raspagem foi realizada, parte do conteúdo é carregado dinamicamente. Dessa forma, foi necessário o uso do Selenium para simular as interações nas páginas, como por exemplo, carregar mais linhas por página e posteriormente aplicar o BeautifulSoup para coletar os dados relevantes.     

#### Dados Coletados e Análises

Os dados analisados foram extraídos no site da [Global Fortune 500](https://fortune.com/global500/).

O Global Fortune 500 é uma classificação das 500 maiores empresas em todo o mundo, de acordo com a medição de sua receita do ano fiscal anterior. Esta lista é compilada e publicada anualmente pela revista Fortune. 

Com base nos dados extraídos foi possível realizar uma análise das informações destas empresas no ano de 2020, tais como: receitas geradas, lucros, número de pessoas empregadas por estas empresas, concentração de quais países/cidades onde as empresas estão instaladas, entre outras.  


#### O projeto se divide assim:

1a Etapa: Instalação e configuração das bibliotecas do Selenium e BeautifulSoup. Aplicação das técnicas de scraping nas páginas utilizando as bibliotecas mencionadas, coleta e estruturação dos dados raspados de cada empresa.

2a Etapa: Criação e validação do pandas dataframe a partir da lista entregue na 1a Etapa, e geração do arquivo "empresas_GlobalFortune500.xlsx" (arquivo disponibilizado no github). 

3a Etapa: Análise exploratória dos dados das empresas.


### Instalação e configuração das bibiliotecas necessárias

###### Selenium

O Selenium é a biblioteca que permitirá realizar as interações na página web, simulando o comportamento de um usuário.

###### WebDriver

WebDrivers são modulos executáveis que rodam no sistema com o browser a ser manipulado. O driver é específico para cada browser, como ChromeDriver para o Chrome/Chromium e o GeckoDriver para Firefox.

Neste projeto foi utilizado o ChromeDriver, disponibilizado no site https://chromedriver.chromium.org/downloads.




In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


Importa o módulo webdriver e realiza as parametrizações iniciais do driver.

In [4]:
from selenium import webdriver

options = webdriver.ChromeOptions()

options.add_argument('--headless') 
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage') 
options.add_argument('--start-maximized') 
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36')
options.add_argument('--ignore-certificate-errors')

driver = webdriver.Chrome(options=options, executable_path=r'C:\Users\bruno\Projetos PUC-RIO\TCC - PUC\TCC_BI_MASTER\chromedriver.exe')

Limpa os eventuais coockies presentes no driver.

In [5]:
driver.delete_all_cookies()

Acessa a página web onde serão coletados os dados.

In [7]:
driver.get("https://fortune.com/global500/2020/search/")

### Scraping unindo o BeautifulSoup e Selenium para coleta dos dados

O código abaixo representa a simulação de um usuário exibindo 100 linhas (empresas) por página e posteriormente visitando página por página até chegar a última possível. A interação foi realizada através das funções disponíveis no Selenium.

Para cada página listada, com as respectivas 100 empresas rankeadas, os dados foram coletados através das funções disponíveis no BeautifulSoup

In [8]:
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

data_list_companies = []

show_onehundred_companies = Select(driver.find_element_by_xpath("//select[@aria-label='rows per page']"))

# setar a exibição de cem linhas por página
show_onehundred_companies.select_by_value('100')

jump_page = driver.find_element_by_xpath("//input[@aria-label='jump to page']")

page = 1

while page < 6:
   
    driver.implicitly_wait(10000)
    page_html = driver.page_source
    soup = BeautifulSoup(page_html,'html.parser')

    companies = soup.find_all('div', attrs={'class':'rt-tr-group'})
    
    # coleta os dados e os sites de cada uma das cem empresas listadas 
    # este site individual por empresa será utilizado mais a frente para a obtenção de mais dados
    for companie in companies:
        data, site = companie.find_all('a'), companie.find('a')      

        data = [ele.text for ele in data]
        site = [site.get('href')]        
        data_complete = data + site

        data_list_companies.append(data_complete)
    
    page = page + 1
    
    # deleta o valor inserido no campo para que um novo valor de paginação seja preenchido
    jump_page.send_keys(Keys.BACKSPACE)    
    jump_page.send_keys(page, Keys.ENTER)    

Imprime para validação dos dados obtidos a primeira e última empresa listadas em cada página.

In [9]:
i = 0
j = 99
page = 1

while page < 6:
    print("   Pág {}. Primeira e última empresa da lista".format(page))
    print("   " , data_list_companies[i][0:2])
    print("   " , data_list_companies[j][0:2])
    print("   _____________________________________\n")
    i = i + 100
    j = j + 100
    page = page + 1

   Pág 1. Primeira e última empresa da lista
    ['1', 'Walmart']
    ['100', 'Dongfeng Motor']
   _____________________________________

   Pág 2. Primeira e última empresa da lista
    ['101', 'Peugeot']
    ['200', 'Energy Transfer']
   _____________________________________

   Pág 3. Primeira e última empresa da lista
    ['201', 'Telefónica']
    ['300', 'Mitsubishi Electric']
   _____________________________________

   Pág 4. Primeira e última empresa da lista
    ['301', 'Jardine Matheson']
    ['400', 'Mazda Motor']
   _____________________________________

   Pág 5. Primeira e última empresa da lista
    ['401', 'Ansteel Group']
    ['500', 'Shanxi Jincheng Anthracite Coal Mining Group']
   _____________________________________



Para cada empresa obtida, além dos dados apresentados em tela, também foram coletados os sites de cada empresa disponibilizados no próprio site da Global Fortune 500. Estes sites também foram raspados, pois contém mais dados, diferentes dos já coletados, que também possuem uma relevância para a análise das empresas.

In [10]:
sites = []

for i in range(500):
    urls = data_list_companies[i][10]
    sites.append(urls)
    
sites[0:10] # exibe os 10 primeiros sites 

['https://fortune.com/company/walmart/global500/',
 'https://fortune.com/company/sinopec-group/global500/',
 'https://fortune.com/company/state-grid/global500/',
 'https://fortune.com/company/china-national-petroleum/global500/',
 'https://fortune.com/company/royal-dutch-shell/global500/',
 'https://fortune.com/company/saudi-aramco/global500/',
 'https://fortune.com/company/volkswagen/global500/',
 'https://fortune.com/company/bp/global500/',
 'https://fortune.com/company/amazon-com/global500/',
 'https://fortune.com/company/toyota-motor/global500/']

Obtenção dos dados disponíveis no site de cada empresa e criação da lista final (data_companies) com todos os dados de todas as empresas que serão posteriormente analisados.

In [11]:
data_companie = []
data_companies = []

i = 0
for site in sites:
    
    driver.get(site)
    page_html = driver.page_source
    
    soup = BeautifulSoup(page_html,'html.parser')    
    data = soup.find_all('div', attrs={'class':'info__value--2AHH7'})    
    data_companie.append(data)
    
    # informações presentes na página inicial onde são listadas todas as empresas
    rank = data_list_companies[i][0]
    name = data_list_companies[i][1]
    revenues_percent = data_list_companies[i][3]
    profits_percent = data_list_companies[i][5]
    assets = data_list_companies[i][6]
    change_rank = data_list_companies[i][8]
    years_global500 = data_list_companies[i][9]
    
    # informações presentes no site individual de cada empresa
    country = data_companie[i][0].text
    headquarters = data_companie[i][1].text
    industry = data_companie[i][2].text
    ceo = data_companie[i][3].text
    companyType = data_companie[i][4].text
    ticker = data_companie[i][5].text
    revenues = data_companie[i][6].text
    profits = data_companie[i][7].text
    marketValue = data_companie[i][8].text
    employees = data_companie[i][9].text
     
    i = i + 1
    
    # inserindo todas as informações presentes na listagem inicial e no site de cada empresa
    data_companies.append((rank, name, country, headquarters, industry, ceo, 
                           companyType, ticker, revenues, revenues_percent, 
                           profits, profits_percent, assets, marketValue, 
                           employees, change_rank, years_global500))      


Finaliza o driver

In [12]:
driver.quit()

### Criação e validação inicial dos dados no dataframe e arquivo excel 

In [13]:
import pandas as pd

df = pd.DataFrame(data_companies, columns=['rank', 'name', 'country', 'headquarters', 'industry',
                                           'ceo','companyType', 'ticker', 'revenues', 'revenues_percent', 
                                           'profits', 'profits_percent', 'assets', 'marketValue', 
                                           'employees','change_rank', 'years_global500'])



Cria o arquivo excel (arquivo disponibilizado no github)

In [13]:
df.to_excel('C:/Users/bruno/Projetos PUC-RIO/TCC - PUC/TCC_BI_MASTER/empresas_GlobalFortune500.xlsx') 

Validação do dataframe

In [14]:
df.shape

(500, 17)

In [15]:
df.head() # as tabelas abaixo se apresentaram desconfiguradas em algumas partes no github

,rank,name,country,headquarters,industry,ceo,companyType,ticker,revenues,revenues_percent,profits,profits_percent,assets,marketValue,employees,change_rank,years_global500
0,1,Walmart,U.S.,"Bentonville, Ark.",General Merchandisers,C. Douglas McMillon,Public,WMT,"$559,151",1.9%,"$13,510",123.1%,"$236,495","$382,642.8","2,300,000",-,26
1,2,Sinopec Group,China,Beijing,Petroleum Refining,Zhang Yuzhuo,Private,,"$407,009",-1.8%,"$6,793.2",16.2%,"$317,515.7",$,"582,648",-,22
2,3,State Grid,China,Beijing,Utilities,Xin Baoan,Private,,"$383,906",-0.8%,"$7,970",-2.5%,"$596,616.3",$,"907,677",2,20
3,4,China National Petroleum,China,Beijing,Petroleum Refining,Li FanRong,Private,,"$379,130",-3.5%,"$4,443.2",95.7%,"$608,085.6",$,"1,344,410",-,20
4,5,Royal Dutch Shell,Netherlands,The Hague,Petroleum Refining,Ben van Beurden,Public,RDS.A,"$352,106",-11.2%,"$15,842",-32.2%,"$404,336","$112,725","83,000",-2,26


In [16]:
df.tail()

,rank,name,country,headquarters,industry,ceo,companyType,ticker,revenues,revenues_percent,profits,profits_percent,assets,marketValue,employees,change_rank,years_global500
495,496,China National Coal Group,China,Beijing,"Mining, Crude-Oil Production",Li Yanjiang,Private,,"$25,846",14%,$307.7,-,"$56,840.6",$,"122,827",-,1
496,497,Paccar,U.S.,"Bellevue, Wash.",Construction and Farm Machinery,R. Preston Feight,Public,PCAR,"$18,728.5",9%,"$1,298.4",8.8%,"$28,361.1","$32,255.8","26,000",-,3
497,498,Thermo Fisher Scientific,U.S.,"Waltham, Mass.","Scientific,Photographic and Control Equipment",Marc N. Casper,Public,TMO,"$32,218",4.9%,"$6,375",25.8%,"$58,381","$179,719.4","84,362",-,1
498,499,Yangquan Coal Industry Group,China,"Yangquan, China","Mining, Crude-Oil Production",Zhai Hong,Private,,"$25,491",-3%,$-81.5,-,"$35,408.8",$,"101,817",-30,8
499,500,Shanxi Jincheng Anthracite Coal Mining Group,China,"Jincheng, China","Mining, Crude-Oil Production",Li Hongshuang,Private,,"$25,386",-1.8%,$22,-58.3%,"$42,256.6",$,"127,336",-18,8


### Análise exploratória dos dados das empresas (em desenvolvimento)

O arquivo gerado foi utilizado como fonte de dados para importação no [Power BI](https://docs.microsoft.com/pt-br/power-bi/) e assim responder as análises propostas na introdução deste material. Os dados carregados foram tratados no editor do Power Query, onde foi realizada a remoção de colunas desnecessárias, extração/substituição de textos e a conversão de tipos de dados. Algumas medidas também foram criadas para a utilização nos diferentes tipos de visualizações dos relatórios. O arquivo [empresas_GlobalFortune500.xlsx](https://github.com/Archettti/TCC_BI_MASTER/blob/main/empresas_GlobalFortune500.xlsx), com todos os passos informados acima, além dos relatórios completos, também está disponibilizado no github.

O primeiro relatório foi sobre a concentração geográfica das 500 maiores empresas do mundo em 2020. Elas estão distribuídas em 33 países. A China e Estados Unidos lideram com 124 e 122 empresas representadas respectivamente, sendo Pequim a cidade que mais abriga estas empresas como sede. No Brasil, existem 07 empresas rankeadas na lista: Petrobras (120); JBS (213); Itáu (216); Bradesco (268); BB (288); CEF (326); e Vale (333). 

<img src="img/relEmpresas.png" width="800" /> 

Ao todo, as maiores empresas do mundo em 2020 geraram  32,7 Tri de dólares em receitas. Boa parte concentrada nos Estados Unidos e China, que juntas possuem 246 empresas das 500 listadas, pouco menos da metade da lista. As duas empresas que mais geraram receita em 2020 foram Walmart (559,15 Bi) e Sinopec (407,01 Bi). 

<img src="img/relReceitas.png" width="800" /> 

Além da receita gerada, as 500 maiores empresas do mundo obtiveram 1,7 Tri de dólares em lucros em 2020 e possuem o valor de mercado no total de 36,02 Tri de dólares. As empresas mais lucrativas e mais valiosas no mercado estão representadas no relatório abaixo.

<img src="img/relLucroMercado.png" width="800" /> 

Juntas, as empresas Fortune Global 500 de 2020 empregaram 70,38 milhões de pessoas em todo o mundo. Mais uma vez China e Estados Unidos lideram por consequência da representatividade de boa parte das empresas na lista.

<img src="img/relEmpregados.png" width="800" /> 

Os cinco tipos de indústrias com maior representatividade no Global Fortune 500 são, nesta ordem: Bancos; Refino de Petróleo; Seguros; Mineiração & Produção de Óleo; e Veículo Motorizados & Peças. A indústria petrolífera é que a mais gerou receita em 2020, o equivalente a 3,19 Tri de dólares. Já as empresas do setor bancário são as que mais empregaram, totalizando 4,78 milhões de pessoas trabalhando neste setor em todo o mundo. 

<img src="img/relIndustrias.png" width="800" /> 

As empresas brasileiras na Global Fortune 500 estão divididas entre as indústrias petrolífera, bancária, alimentícia e de mineiração. Cada setor possui uma grande contribuição na geração de receita e empregos ao país. 


<img src="img/empresasBrasil.gif" width="800" /> 